In [ ]:
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    MeshcatVisualizer,
    Parser,
    Simulator,
    StartMeshcat,
    MultibodyPlant,
    InverseDynamicsController,
    Diagram,
    LeafSystem,
    Integrator,
    JacobianWrtVariable
)
import numpy as np
from manipulation.utils import RenderDiagram

In [ ]:
meshcat = StartMeshcat()
running_as_notebook = True


In [ ]:
plant = MultibodyPlant(time_step=1e-2)
# Note that we parse into both the plant and the scene_graph here.
Parser(plant).AddModelsFromUrl(
        "file:///Users/raghav/Documents/projects/robot_arm/low_cost_robot/drake_setup/low-cost-arm.urdf"
)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"))
plant.Finalize()

In [ ]:
context = plant.CreateDefaultContext()
print(context)

In [ ]:
plant.SetPositions(context, [1,1,1,1,1, 1])
plant.GetJointByName("revolute_2").set_angle(context, 2)
print(context)

In [ ]:
# plant.get_actuation_input_port().FixValue(context, np.zeros(5))
input_port = plant.get_geometry_query_input_port()

In [ ]:
simulator = Simulator(plant, context)
simulator.AdvanceTo(5.0)
print(context)

In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()
builder = DiagramBuilder()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
# Note that we parse into both the plant and the scene_graph here.
Parser(plant, scene_graph).AddModelsFromUrl(
     "file:///Users/raghav/Documents/projects/robot_arm/low_cost_robot/drake_setup/low-cost-arm.urdf"
)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"))
plant.Finalize()

# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
diagram.set_name("plant and scene_graph")


In [ ]:
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

In [ ]:
RenderDiagram(diagram)

In [ ]:
plant_context = plant.GetMyMutableContextFromRoot(context)
plant.GetPositions(plant_context)

In [ ]:
plant_context = plant.GetMyMutableContextFromRoot(context)
plant.SetPositions(plant_context, [0.5, 0, 0, 0, -1])
# plant.get_actuation_input_port().FixValue(plant_context, np.zeros(5)) # is this line doing anything for low cost robot?
# print(context)
print(plant.GetPositions(plant_context))

# re-render
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)
# RenderDiagram(diagram)

In [ ]:
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0 if running_as_notebook else 0.1)

In [ ]:
def animation_demo():
    builder = DiagramBuilder()

    # Adds both MultibodyPlant and the SceneGraph, and wires them together.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    # Note that we parse into both the plant and the scene_graph here.
    Parser(plant, scene_graph).AddModelsFromUrl(
            "file:///Users/raghav/Documents/projects/robot_arm/low_cost_robot/drake_setup/low-cost-arm.urdf"
    )
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"))
    plant.Finalize()

    # Adds the MeshcatVisualizer and wires it to the SceneGraph.
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    print("Positions", plant.GetPositions(plant_context))
    plant.SetPositions(plant_context, [0, 0, 0, 0, 0])
    plant.get_actuation_input_port().FixValue(plant_context, np.zeros(5))

    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)

    # Record things if needed
    # meshcat.StartRecording()
    simulator.AdvanceTo(5.0 if running_as_notebook else 0.1)
    # meshcat.StopRecording()
    # meshcat.PublishRecording()


animation_demo()


## Animation with Robot Sample Driver

In [ ]:
# Control
builder = DiagramBuilder()
meshcat.DeleteRecording()

# Adds both MultibodyPlant and the SceneGraph, and wires them together.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
# Note that we parse into both the plant and the scene_graph here.
robot_model = Parser(plant, scene_graph).AddModelsFromUrl(
        "file:///Users/raghav/Documents/projects/robot_arm/low_cost_robot/drake_setup/low-cost-arm.urdf"
)[0]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"))
plant.Finalize()

# Adds the MeshcatVisualizer and wires it to the SceneGraph.
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
kp = [100] * plant.num_positions()
ki = [1] * plant.num_positions()
kd = [20] * plant.num_positions()
static_controller = builder.AddSystem(InverseDynamicsController(plant, kp, ki, kd, False))
static_controller.set_name("static_controller")
builder.Connect(
    plant.get_state_output_port(robot_model),
    static_controller.get_input_port_estimated_state(),
)
builder.Connect(
    static_controller.get_output_port_control(), plant.get_actuation_input_port()
)
diagram = builder.Build()
diagram.set_name("with iiwa controller")

In [ ]:
RenderDiagram(diagram, max_depth=1)

In [ ]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = np.array([-1.57, 0.1, 0, -1.2, 0])
x0 = np.hstack((q0, 0 * q0))
plant.SetPositions(plant_context, q0)
static_controller.GetInputPort("desired_state").FixValue(
    static_controller.GetMyMutableContextFromRoot(context), x0
)
print(context)
plant.get_state_output_port(robot_model).Eval(plant_context)[:5]

In [ ]:
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0 if running_as_notebook else 0.1);

At this point, the robot should be stable. You can play around with `q0` below.

In [ ]:
# play around with q0
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = np.array([0 , -np.pi / 2, 0, 0, -np.pi / 2])
x0 = np.hstack((q0, 0 * q0))
plant.SetPositions(plant_context, np.zeros(5))
static_controller.GetInputPort("desired_state").FixValue(
    static_controller.GetMyMutableContextFromRoot(context), x0
)
simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(1.0)

In [ ]:
class PseudoInverseController(LeafSystem):
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._low_cost_robot = plant.GetModelInstanceByName("low-cost-arm")
        self._G = plant.GetBodyByName("gripper-static-motor_v2_1").body_frame()
        self._W = plant.world_frame()

        self.DeclareVectorInputPort("low_cost_robot.position", 5)
        self.DeclareVectorOutputPort("low_cost_robot.velocity", 5, self.CalcOutput)

    def CalcOutput(self, context, output):
        q = self.get_input_port().Eval(context)
        self._plant.SetPositions(self._plant_context, self._low_cost_robot, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G,
            [0, 0, 0],
            self._W,
            self._W,
        )
        # J_G = J_G[:, 0:]  # Ignore gripper terms

        V_G_desired = np.array(
            [
                0,  # rotation about x
                0,  # rotation about y
                0,  # rotation about z
                0.3,  # x
                0,  # y
                0.3, # z
            ]
        ) 
        v = np.linalg.pinv(J_G).dot(V_G_desired)
        output.SetFromVector(v)

# Have separate classes for this so you can separate the integrator from the velocity/ position command.
class ExtractPositionFromState(LeafSystem):
    def __init__(self, ):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("estimated_state", 10)
        self.DeclareVectorOutputPort("position", 5, self.CalcOutput)

    def CalcOutput(self, context, output):
        full_state = self.get_input_port().Eval(context)
        output.SetFromVector(full_state[:5])

class ExtractStateFromPositionAndVelocity(LeafSystem):
    def __init__(self, ):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("low_cost_robot.desired_position", 5)
        self.DeclareVectorInputPort("low_cost_robot.desired_velocity", 5)
        self.DeclareVectorOutputPort("low_cost_robot.desired_state", 10, self.CalcOutput)

    def CalcOutput(self, context, output):
        position = self.get_input_port(0).Eval(context)
        velocity = self.get_input_port(1).Eval(context)
        # print(position, velocity)
        output.SetFromVector(np.hstack((position, velocity)))
        

def jacobian_controller_example():
    # Control
    builder = DiagramBuilder()
    
    # Adds both MultibodyPlant and the SceneGraph, and wires them together.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-2)
    # Note that we parse into both the plant and the scene_graph here.
    robot_model = Parser(plant, scene_graph).AddModelsFromUrl(
            "file:///Users/raghav/Documents/projects/robot_arm/low_cost_robot/drake_setup/low-cost-arm.urdf"
    )[0]
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base_link"))
    plant.Finalize()
    
    # Adds the MeshcatVisualizer and wires it to the SceneGraph.
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    controller = builder.AddSystem(PseudoInverseController(plant))
    integrator = builder.AddSystem(Integrator(5))
    pos_from_state = builder.AddSystem(ExtractPositionFromState())
    # state_from_pos_vel = builder.AddSystem(ExtractStateFromPositionAndVelocity())
    


    builder.Connect(controller.get_output_port(), integrator.get_input_port())
    builder.Connect(integrator.get_output_port(), plant.get_actuation_input_port())
    builder.Connect(plant.get_state_output_port(robot_model), pos_from_state.get_input_port())
    builder.Connect(pos_from_state.get_output_port(), controller.get_input_port())
    
    diagram = builder.Build()
    diagram.set_name("with iiwa controller")

    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    
    
    RenderDiagram(diagram)
    integrator.set_integral_value(
        integrator.GetMyContextFromRoot(context),
        plant.GetPositions(
            plant.GetMyContextFromRoot(context),
            plant.GetModelInstanceByName("low-cost-arm"),
        ),
    )

    

    # meshcat.StartRecording()
    simulator.AdvanceTo(100 if running_as_notebook else 0.1)
    meshcat.PublishRecording()
jacobian_controller_example()

# Turns out this just doesn't make sense unless I can figure out how to give pose commands to the sim.